<a href="https://colab.research.google.com/github/sridevibonthu/TSAI_END_P1/blob/main/Session%2007/Sentiment_Analysis_on_stanford_data_using_LSTM_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0 TorchText

## Dataset Preview

Your first step to deep learning in NLP. We will be mostly using PyTorch. Just like torchvision, PyTorch provides an official library, torchtext, for handling text-processing pipelines. 

We will be using previous session tweet dataset. Let's just preview the dataset.

In [45]:
import sqlite3
import csv
import sys

with open('datasetSentences.txt') as infile:
    infile.readline()
    sentences = dict(line.strip().split('\t') for line in infile if len(line) > 0)

with open('dictionary.txt') as infile:
    infile.readline()
    dictionary = dict(line.strip().split('|')[::-1] for line in infile if len(line) > 0)

with open('sentiment_labels.txt') as infile:
    infile.readline()
    sentiments = dict(line.strip().strip('!').split('|') for line in infile if len(line) > 0)

In [46]:


conn = sqlite3.connect(':memory:')
conn.execute('CREATE TABLE sentences (id LONG PRIMARY KEY, sentence TEXT)')
conn.execute('CREATE TABLE dictionary (id LONG PRIMARY KEY, phrase TEXT)')
conn.execute('CREATE TABLE sentiments (phrase_id LONG PRIMARY KEY, sentiment {})'.format('FLOAT'))
conn.execute('CREATE INDEX sentences_sentence_idx ON sentences (sentence);')
conn.execute('CREATE INDEX dictionary_phrase_idx ON dictionary (phrase);')

conn.executemany('INSERT INTO sentences VALUES (?, ?)', sentences.items())
conn.executemany('INSERT INTO dictionary VALUES (?, ?)', dictionary.items())
conn.executemany('INSERT INTO sentiments VALUES (?, ?)', sentiments.items())


conn.commit()
crs = conn.cursor()


In [47]:

crs.execute('''
        SELECT
            sentence,
            sentiment
        FROM sentences 
            JOIN dictionary ON sentence = phrase
            JOIN sentiments ON dictionary.id = sentiments.phrase_id
        ''')


dataset = crs.fetchall()

with open('data.csv', 'w') as outfile:
  writer = csv.writer(outfile)
  writer.writerow(['sentence', 'sentiment'])
  for row in dataset:
    writer.writerow(row[:])


In [48]:
import pandas as pd
df = pd.read_csv('data.csv')
df.head()

,sentence,sentiment
0,... a bland murder-on-campus yawner .,0.013889
1,... a hollow joke told by a cinematic gymnast ...,0.263890
2,... the picture 's cleverness is ironically mu...,0.472220
3,classic cinema served up with heart and humor,0.875000
4,"entertaining enough , but nothing new",0.319440


In [49]:
def score_to_label(score):
  if score <= 0.2:
    return 0
  elif score <= 0.4:
    return 1
  elif score <= 0.6:
    return 2
  elif score <= 0.8:
    return 3
  else:
    return 4

df['label'] = df.apply(lambda row: score_to_label(row['sentiment']), axis=1)
df.head()


,sentence,sentiment,label
0,... a bland murder-on-campus yawner .,0.013889,0
1,... a hollow joke told by a cinematic gymnast ...,0.263890,1
2,... the picture 's cleverness is ironically mu...,0.472220,2
3,classic cinema served up with heart and humor,0.875000,4
4,"entertaining enough , but nothing new",0.319440,1


In [50]:
print(df.shape)

(11286, 3)


In [51]:
df.label.value_counts()

1    2971
3    2966
2    2144
4    1773
0    1432
Name: label, dtype: int64

## Defining Fields

Now we shall be defining LABEL as a LabelField, which is a subclass of Field that sets sequen tial to False (as it’s our numerical category class). TWEET is a standard Field object, where we have decided to use the spaCy tokenizer and convert all the text to lower‐ case.

In [52]:
# Import Library
import random
import torch, torchtext
from torchtext import data 

# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [53]:
Sentence = data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

Having defined those fields, we now need to produce a list that maps them onto the list of rows that are in the CSV:

In [54]:
fields = [('sentence', Sentence),('label',Label)]

Armed with our declared fields, lets convert from pandas to list to torchtext. We could also use TabularDataset to apply that definition to the CSV directly but showing an alternative approach too.

In [55]:
!pip install google_trans_new


In [56]:
# Augmentation

import random
import google_trans_new

from google_trans_new import google_translator

translator = google_translator()



def random_insertion(sentence, n): 
    words = remove_stopwords(sentence) 
    for _ in range(n):
        new_synonym = get_synonyms(random.choice(words))
        sentence.insert(randrange(len(sentence)+1), new_synonym) 
    return sentence

def random_deletion(words, p=0.5): 
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)] 
    else:
        return remaining

def random_swap(sentence, n=5): 
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

def googletranslate(text):
  tempfr = translator.translate(text, lang_tgt='fr')
  return translator.translate(tempfr, lang_src='fr', lang_tgt='en')

In [64]:
def augment(text):
  rate = random.random()
  #print(rate, end=' ')
  if rate<0.2:
    text = random_deletion(text)
  elif rate<0.45:
    text = random_swap(text)
  elif rate<0.5:
    text = list(googletranslate(text).split(' '))
  return text

In [58]:
example = [data.Example.fromlist([df.sentence[i],df.label[i]], fields) for i in range(df.shape[0])] 

In [59]:
# Creating dataset
#twitterDataset = data.TabularDataset(path="tweets.csv", format="CSV", fields=fields, skip_header=True)

SSTDataset = data.Dataset(example, fields)

Finally, we can split into training, testing, and validation sets by using the split() method:

In [60]:
(train, valid) = SSTDataset.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

In [61]:
(len(train), len(valid))

(9593, 1693)

### Applying Augmentations on train data

In [65]:
for i in range(len(train)):
  train.examples[i].sentence = augment(train.examples[i].sentence)

0.038551839337380045 0.43406813885223416 0.512430346269225 0.4959772387795468 0.5751589252191758 0.376925185796109 0.9882314343429207 0.8135770826367814 0.6539875189422611 0.6497514223996674 0.06103923986925708 0.2681270418170997 0.8993664739961356 0.7285757210495236 0.36157193567736523 0.08622779602940023 0.4981113542540818 0.8572114404307897 0.38559491997321416 0.4087027081284952 0.7062107230548309 0.9649980519048675 0.6696522923814167 0.9588198907667034 0.22193521518558235 0.09598781933788703 0.11639444029615442 0.6995228502096925 0.7262794510988779 0.15406074081636434 0.9980330203627096 0.4191558681221669 0.3328039598824557 0.23662932806680503 0.637445643906559 0.9484322978669255 0.30293227647185006 0.26094394852033076 0.8332588705095431 0.3083412759086108 0.5756726595705399 0.6315693591472082 0.561868052711989 0.5427101675293277 0.9536339276027482 0.008944658006400097 0.9217040765502291 0.0885761893628414 0.4424456390503304 0.07744665787745653 0.03803025692649986 0.025646786067222

In [ ]:
* An example from training data

In [66]:
vars(train.examples[12])

{'label': 0,
 'sentence': ['The',
  'attempt',
  'to',
  'build',
  'up',
  'a',
  'pressure',
  'cooker',
  'of',
  'horrified',
  'awe',
  'emerges',
  'from',
  'the',
  'simple',
  'fact',
  'that',
  'the',
  'movie',
  'has',
  'virtually',
  'nothing',
  'to',
  'show',
  '.']}

## Building Vocabulary

At this point we would have built a one-hot encoding of each word that is present in the dataset—a rather tedious process. Thankfully, torchtext will do this for us, and will also allow a max_size parameter to be passed in to limit the vocabu‐ lary to the most common words. This is normally done to prevent the construction of a huge, memory-hungry model. We don’t want our GPUs too overwhelmed, after all. 

Let’s limit the vocabulary to a maximum of 5000 words in our training set:


In [67]:
Sentence.build_vocab(train)
Label.build_vocab(train)

By default, torchtext will add two more special tokens, <unk> for unknown words and <pad>, a padding token that will be used to pad all our text to roughly the same size to help with efficient batching on the GPU.

In [68]:
print('Size of input vocab : ', len(Sentence.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Sentence.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  19408
Size of label vocab :  5
Top 10 words appreared repeatedly : [('.', 7736), (',', 6925), ('the', 5885), ('of', 4356), ('a', 4352), ('and', 4247), ('to', 2992), ('-', 2659), ('is', 2482), ("','", 2447)]
Labels :  defaultdict(<function _default_unk_index at 0x7faa6c459a60>, {1: 0, 3: 1, 2: 2, 4: 3, 0: 4})


**Lots of stopwords!!**

Now we need to create a data loader to feed into our training loop. Torchtext provides the BucketIterator method that will produce what it calls a Batch, which is almost, but not quite, like the data loader we used on images.

But at first declare the device we are using.

In [69]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [70]:
train_iterator, valid_iterator = data.BucketIterator.splits((train, valid), batch_size = 32, 
                                                            sort_key = lambda x: len(x.sentence),
                                                            sort_within_batch=True, device = device)

Save the vocabulary for later use

In [71]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Sentence.vocab.stoi, tokens)

## Defining Our Model

We use the Embedding and LSTM modules in PyTorch to build a simple model for classifying tweets.

In this model we create three layers. 
1. First, the words in our tweets are pushed into an Embedding layer, which we have established as a 300-dimensional vector embedding. 
2. That’s then fed into a 2 stacked-LSTMs with 100 hidden features (again, we’re compressing down from the 300-dimensional input like we did with images). We are using 2 LSTMs for using the dropout.
3. Finally, the output of the LSTM (the final hidden state after processing the incoming tweet) is pushed through a standard fully connected layer with three outputs to correspond to our three possible classes (negative, positive, or neutral).

In [113]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True, bidirectional = True
                           )
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [114]:
# Define hyperparameters
size_of_vocab = len(Sentence.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 5
num_layers = 4
dropout = 0.2

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [115]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(19408, 300)
  (encoder): LSTM(300, 100, num_layers=4, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=100, out_features=5, bias=True)
)
The model has 6,869,305 trainable parameters



## Model Training and Evaluation

First define the optimizer and loss functions

In [116]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

The main thing to be aware of in this new training loop is that we have to reference `batch.tweets` and `batch.labels` to get the particular fields we’re interested in; they don’t fall out quite as nicely from the enumerator as they do in torchvision.

**Training Loop**

In [117]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        sentence, sentence_lengths = batch.sentence   
        
        # convert to 1D tensor
        predictions = model(sentence, sentence_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.label)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.label)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Evaluation Loop**

In [118]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            sentence, sentence_lengths = batch.sentence
            
            
            # convert to 1d tensor
            predictions = model(sentence, sentence_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Let's Train and Evaluate**

In [119]:
N_EPOCHS = 20
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	Train Loss: 1.589 | Train Acc: 25.18%
	 Val. Loss: 1.577 |  Val. Acc: 25.46% 

	Train Loss: 1.572 | Train Acc: 30.63%
	 Val. Loss: 1.566 |  Val. Acc: 30.69% 

	Train Loss: 1.544 | Train Acc: 34.81%
	 Val. Loss: 1.550 |  Val. Acc: 33.16% 

	Train Loss: 1.505 | Train Acc: 39.52%
	 Val. Loss: 1.543 |  Val. Acc: 34.17% 

	Train Loss: 1.467 | Train Acc: 43.69%
	 Val. Loss: 1.541 |  Val. Acc: 34.53% 

	Train Loss: 1.432 | Train Acc: 47.80%
	 Val. Loss: 1.542 |  Val. Acc: 33.65% 

	Train Loss: 1.401 | Train Acc: 51.47%
	 Val. Loss: 1.543 |  Val. Acc: 33.70% 

	Train Loss: 1.366 | Train Acc: 55.39%
	 Val. Loss: 1.550 |  Val. Acc: 32.35% 

	Train Loss: 1.335 | Train Acc: 58.63%
	 Val. Loss: 1.552 |  Val. Acc: 32.88% 

	Train Loss: 1.307 | Train Acc: 61.75%
	 Val. Loss: 1.551 |  Val. Acc: 32.64% 

	Train Loss: 1.281 | Train Acc: 64.16%
	 Val. Loss: 1.559 |  Val. Acc: 32.12% 

	Train Loss: 1.251 | Train Acc: 67.66%
	 Val. Loss: 1.561 |  Val. Acc: 31.40% 

	Train Loss: 1.224 | Train Acc: 70.40%
	

## Model Testing

In [120]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_tweet(tweet):
    
    categories = { 0: "Very Negative",
        1: "Negative",
        2: "Neutral",
        3: "Positive",
        4: "Very Positive" }
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(tweet)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [121]:
classify_tweet("I'll probably have to watch it again to understand what happens.")

'Negative'

## Discussion on Data Augmentation Techniques 

You might wonder exactly how you can augment text data. After all, you can’t really flip it horizontally as you can an image! :D 

In contrast to data augmentation in images, augmentation techniques on data is very specific to final product you are building. As its general usage on any type of textual data doesn't provides a significant performance boost, that's why unlike torchvision, torchtext doesn’t offer a augmentation pipeline. Due to powerful models as transformers, augmentation tecnhiques are not so preferred now-a-days. But its better to know about some techniques with text that will provide your model with a little more information for training. 

### Synonym Replacement

First, you could replace words in the sentence with synonyms, like so:

    The dog slept on the mat

could become

    The dog slept on the rug

Aside from the dog's insistence that a rug is much softer than a mat, the meaning of the sentence hasn’t changed. But mat and rug will be mapped to different indices in the vocabulary, so the model will learn that the two sentences map to the same label, and hopefully that there’s a connection between those two words, as everything else in the sentences is the same.

### Random Insertion
A random insertion technique looks at a sentence and then randomly inserts synonyms of existing non-stopwords into the sentence n times. Assuming you have a way of getting a synonym of a word and a way of eliminating stopwords (common words such as and, it, the, etc.), shown, but not implemented, in this function via get_synonyms() and get_stopwords(), an implementation of this would be as follows:


In [ ]:
def random_insertion(sentence, n): 
    words = remove_stopwords(sentence) 
    for _ in range(n):
        new_synonym = get_synonyms(random.choice(words))
        sentence.insert(randrange(len(sentence)+1), new_synonym) 
    return sentence

## Random Deletion
As the name suggests, random deletion deletes words from a sentence. Given a probability parameter p, it will go through the sentence and decide whether to delete a word or not based on that random probability. Consider of it as pixel dropouts while treating images.

In [ ]:
def random_deletion(words, p=0.5): 
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)] 
    else:
        return remaining

### Random Swap
The random swap augmentation takes a sentence and then swaps words within it n times, with each iteration working on the previously swapped sentence. Here we sample two random numbers based on the length of the sentence, and then just keep swapping until we hit n.

In [ ]:
def random_swap(sentence, n=5): 
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

For more on this please go through this [paper](https://arxiv.org/pdf/1901.11196.pdf).

### Back Translation

Another popular approach for augmenting text datasets is back translation. This involves translating a sentence from our target language into one or more other languages and then translating all of them back to the original language. We can use the Python library googletrans for this purpose. 

In [ ]:
!pip install google_trans_new


In [ ]:
import random
import google_trans_new

from google_trans_new import google_translator

translator = google_translator()
sentence = ['The dog slept on the rug', 'This is good coffee']

result = translator.translate(sentence,lang_tgt='fr')
print(result)
translations_en_random = translator.translate(result, lang_src='fr', lang_tgt='en') 
print(translations_en_random)
# en_text = [t for t in translations_en_random]
# print(en_text)

In [ ]:
googletranslate("The cat slept on the rug")